In [12]:
import datetime
import requests
import csv
import sys
import json
import time
import re
import os

In [13]:
# set IO file names
breweries = "breweries.csv"
geocoded_breweries = "geocoded_breweries.csv"

In [15]:
outfile = open(geocoded_breweries, "w")
with open(breweries, 'r') as csvfile:
    reader = csv.reader(csvfile, skipinitialspace=True)
    next(reader, None)
    for row in sorted(reader, key=lambda x: int(x[0])):
        address = ' '.join(row[1:]).lower()
        url = 'https://api.mapanything.io/services/core/geocoding/v2?address={}'.format(address)
        headers = {
            'Accept-Encoding': 'application/gzip',
            'Content-Type': 'application/json',
            'x-api-key': os.getenv("MAPANYTHING_APIKEY")
        }
        response = requests.request('GET', url, headers=headers, allow_redirects=True)
        response = response.json()
        if 'data' in response:
            outfile.write(
                str(row[0]) + "," +
                str(response['data']['position']['lat']) + "," +
                str(response['data']['position']['lng']) + "," +
                str(row[1].lower()) + "," + 
                str(response['data']['fullAddress']) +"\n")
        else:
            outfile.write("0.0, 0.0, {}".format(address) + "\n")
outfile.close()

In [28]:
# current times
utc_offset_sec = time.altzone if time.localtime().tm_isdst else time.timezone
utc_offset = datetime.timedelta(seconds=-utc_offset_sec)
now = datetime.datetime.now().replace(tzinfo=datetime.timezone(offset=utc_offset))
now = now - datetime.timedelta(hours=1)
later = now + datetime.timedelta(hours=8)

# linger times in hours
default_linger = 0.5

# appointment times
time_format = "%Y-%m-%d %H:%M:%S"
gametime = "2020-01-25 15:45:00"
buzzertime = "2020-01-25 18:15:00"
arrive = datetime.datetime.strptime(gametime, time_format).isoformat()
depart = datetime.datetime.strptime(buzzertime, time_format).isoformat()

In [29]:
with open(geocoded_breweries, 'r') as csvfile:
    reader = csv.reader(csvfile, skipinitialspace=True)
    locations = []
    orders = []
    for row in reader:
        locations.append({
            "latitude": float(row[1]),
            "longitude": float(row[2]),
            "location_id": row[3]
        })
        if "home" not in row[3]:
            order = {
                "order_id": row[3],
                "location_id": row[3],
                "duration": default_linger*3600,
                "priority": int(row[0])
            }
            # TODO: add time windows to orders
            if 'elkmont' in row[3]:
                appointment = [
                    {
                        "appointment_start": arrive,
                        "appointment_end": depart
                    }
                ]
                order['appointments'] = appointment
                order['duration'] = 2.5*3600
            orders.append(order)

In [30]:
# blank payload
payload = {}

# breweries to visit
payload['locations'] = locations

# places we actually want to visit (all of them)
payload['orders'] = orders

# how we'll move around
payload['vehicles'] = [
    {
        "vehicle_id": "uber",
        "type": "car",
        "shifts": [
            {
                "shift_start": now.isoformat(),
                "shift_end": later.isoformat(),
                "shift_id": "crawl_shift",
                
                "start_location_id": "home",
                "end_location_id": "home"
            }
        ]
    }
]

# constraints
payload['constraints'] = [
    {
        "constraint_type": "travel_time",
        "constraint_name": "Minimize travel time",
        "violation_increment": 1,
        "penalty_per_violation": 1,
        "max_travel_time_seconds": 0
    },
    {
        "constraint_type": "scheduled_appointment",
        "constraint_name": "Meet scheduled appointments",
        "violation_increment": 1,
        "penalty_per_violation": 10000,
    },
    {
        "constraint_type": "visit_range",
        "constraint_name": "visit as many orders as possible",
        "penalty_per_violation": 100,
        "violation_increment": 1
    }
]

In [31]:
url = 'https://public-api.mapanything.io/mare/routing/sync'
headers = {
    'Accept-Encoding': 'application/gzip',
    'Content-Type': 'application/json',
    'x-api-key': os.getenv("MAPANYTHING_APIKEY")
}
response = requests.request('POST', url, headers=headers, data=json.dumps(payload), allow_redirects=True)
response = response.json()

In [32]:
with open('request.json', 'w') as f:
    json.dump(payload, f)
with open('response.json', 'w') as f:
    json.dump(response, f)

In [33]:
!pbcopy < request.json